In [1]:
from sklearn.linear_model import SGDClassifier
import numpy as np

## Client Update


In [4]:
def client_update(init_weights, epochs, batch_size, features, labels):
    """
    Given the previous weights from the server, it does updates on the model
    and returns the new set of weights

    init_weights: weights to initialize the training with 
        ex: [coef1, coef2, ..., coefn, intercept]
    epochs: number of epochs to run the training for 
    batch_size: the size of each batch of data while training 
    features: a 2D array containing features for each sample 
        ex: [[feature1, feature2], [feature1, feature2], ...]
    label: an array containing the labels for the corresponding sample in "features"
        ex: [label1, label2, ...]
    """
    
    # split the data into batches by given batch_size
    # TODO: need to ensure that a batch doesn't just contain 1 label
    batches_features = []
    batches_labels = []
    
    for i in range(0, len(features), batch_size):
      batches_features.append(features[i:i + batch_size])
      batches_labels.append(labels[i:i + batch_size])
      # TODO: merge the features and labels into one tuple?
    
    print("Features: ", batches_features)
    print("Labels: ", batches_labels)

    weights = list(init_weights)
    
    # set max_iter to 1 so that each .fit() call only does one training step
    classifier = SGDClassifier(loss="hinge", penalty="l2", max_iter=1)

    for epoch in range(epochs):
        for i in range(len(batches_features)):
            print(batches_features[i], batches_labels[i], weights)
            
            classifier.fit(batches_features[i],
                           batches_labels[i],
                           coef_init=weights[:-1],
                           intercept_init=weights[-1])
            
            # update the weights so for the next batch the new ones are used
            weights = np.append(classifier.coef_[0], classifier.intercept_)
            
    return weights

# Example usage

# this data will be provided by the server
features = [[1, 4, 3], [0, 2, 2], [1, 4, 0], [0, 5, 3], [1, 2, 1], [0, 2, 9]]
labels = [1, 0, 1, 0, 1, 0]
weights = [29., 0., 0., -9]
epochs = 3
batch_size = 3

new_weights = client_update(weights, epochs, batch_size, features, labels)
print(new_weights)

Features:  [[[1, 4, 3], [0, 2, 2], [1, 4, 0]], [[0, 5, 3], [1, 2, 1], [0, 2, 9]]]
Labels:  [[1, 0, 1], [0, 1, 0]]
[[1, 4, 3], [0, 2, 2], [1, 4, 0]] [1, 0, 1] [29.0, 0.0, 0.0, -9]
[[0, 5, 3], [1, 2, 1], [0, 2, 9]] [0, 1, 0] [28.91317365  0.          0.         -9.        ]
[[1, 4, 3], [0, 2, 2], [1, 4, 0]] [1, 0, 1] [28.82660726  0.          0.         -9.        ]
[[0, 5, 3], [1, 2, 1], [0, 2, 9]] [0, 1, 0] [28.74030006  0.          0.         -9.        ]
[[1, 4, 3], [0, 2, 2], [1, 4, 0]] [1, 0, 1] [28.65425125  0.          0.         -9.        ]
[[0, 5, 3], [1, 2, 1], [0, 2, 9]] [0, 1, 0] [28.56846008  0.          0.         -9.        ]
[28.48292577  0.          0.         -9.        ]


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/

## Server Update

In [ ]:
import numpy as np

# load data

clients = 10
data_number = 30
features = np.random.randint(10,size=(clients,data_number,3))
labels = np.random.randint(2,size=(100,30))
epoch = 10
batch_size = 20

In [ ]:
from sklearn.linear_model import SGDClassifier
import random
import numpy as np

def append(list, element):
    """
    helper function to append array into array in numpy
    """
    return np.concatenate((list, [element])) if list is not None else [element]

def server():
    
    """
    Calls clientUpdate to get the updated weights from clients, and applies Federated
    Averaging Algorithm to update the weight on server side
    
    """

    # initialize the weight
    w = [0,0,0,0]
    # number of clients
    K = 10
    # fraction of clients
    C = 0.5
    # number of rounds run on the server
    round_num = 10

    # use to generate n_k so that the sum of n_k equals to n

    for i in range(round_num):
        # calculate the number of clients used in this round
        m = max(int(K*C),1)
        # random set of m clients
        S = np.array(random.sample(range(clients), clients))
        
        num_samples = []
            
        # grab all the weights from clients
        client_weights = None   
        for i in S:
            client_feature = features[i]
            client_label = labels[i]
            client_w = np.random.randint(low = 0, high = 10, size=4)
            client_weights = append(client_weights,client_w)
            
            num_samples.append(len(client_feature))

        # calculate the new server weight based on new weights coming from client
        new_w = np.zeros(4)
        for i in range(len(client_weights)):
            current_weight = client_weights[i]
            n_k = len(features[i])
            added_w = [value*(n_k)/sum(num_samples) for value in current_weight]
            new_w = np.add(new_w,added_w)
        
        # update the server weight to newly calculated weight
        w = new_w
        print(w)
        
server()